In [1]:
import sys #ติดตั้งไลบรารี่
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install jupyter_dash
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install dash
!{sys.executable} -m pip install dash-bootstrap-components

In [2]:
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import pandas as pd

In [3]:
#อ่านค่าไฟล์ฐานข้อมูลด้วย pandas เก็บไว้ในตัวแปร
df = pd.read_csv('https://raw.githubusercontent.com/artist0123/ProjectPSIT/main/master.csv', engine = 'python', sep = ',')
code_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

In [4]:
#จัดกลุ่ม Data Frame ตามประเทศและปี
df_country = df.groupby(['Country', 'year']).sum().reset_index()
df_country.replace(0, 1, inplace=True)

In [5]:
#merge Data Frame ประเทศและรหัสประเทศเพื่อนำไปใช้แสดงใน Choropleth map (แผนที่)
df_country_code = df_country.merge(code_df, left_on='Country', right_on='COUNTRY', how='left') 

In [6]:
#จัด Data Frame ตามปี แล้วหาผลรวมของข้อมูลในแต่ละ column เพื่อนำไปใช้ใน Choropleth map
df_year = df.groupby('year').sum()

In [7]:
#จัด Data Frame ตามเพศและช่วงอายุ เพื่อนำไปใช้ในแผนภูมิแท่ง
df_chocolate = df.groupby(['sex', 'age'], as_index=False).sum()

In [8]:
#จัด Data Frame ตามประเทศ เพื่อนำไปใช้หาประเทศที่มีจำนวนการฆ่าตัวตายมากที่สุด
df_suicideno = df.groupby('Country', as_index=False).sum()

In [9]:
#จัด Data Frame ตามช่วงอายุ เพื่อนำไปใช้หาช่วงอายุที่มีจำนวนการฆ่าตัวตายมากที่สุด
df_age = df.groupby('age', as_index=False).sum()

In [10]:
df_suicideno.loc[df_suicideno['suicides_no'].idxmax()] #หาประเทศที่มีจำนวน suicides_no มากที่สุด

Country               Russian Federation
year                              648648
suicides_no                      1209742
population                    3690802620
suicides/100k pop                11305.1
HDI for year                           0
gdp_per_capita ($)               2112096
Name: 75, dtype: object

In [11]:
df_age.loc[df_age['suicides_no'].idxmax()] #หาช่วงอายุที่มีจำนวน suicides_no มากที่สุด

age                   35-54 years
year                      9289920
suicides_no               2452141
population            14375888123
suicides/100k pop           69386
HDI for year              1082.58
gdp_per_capita ($)       78341054
Name: 2, dtype: object

In [12]:
df_suicideno.loc[df_suicideno['Country']=='Thailand'] #หาข้อมูลของประเทศไทย

,Country,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
91,Thailand,668508,110643,1584800281,2362.67,79.98,1193102


In [13]:
 # ส่วน Layout ใช้ Component ของ Dash และ Dash Bootstrap เพื่อมาแสดงเป็น Dashboard 
YEARS = [x for x in range(1985, 2017)]
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#Choropleth map
card1 = [dbc.CardHeader(children=[
                dcc.Graph(id='world-choropleth'),
                        ]),#CardHeader
                        dbc.CardBody(children=[
                            dcc.Slider(
                            id='year-slider',
                            min=min(YEARS),
                            max=max(YEARS),
                            value=min(YEARS),
                            marks={i:"%d" % i for i in YEARS},
                            ),
                            html.P(#ข้อความใน card
                                "นี่คือกราฟแผนที่แสดงการฆ่าตัวตายของทั่วโลกตั้งแต่ปี 1985-2016", 
                                className="card-text",
                            ),
                            html.P(
                                "ท่านสามารถเลือกดูปีที่ต้องการได้โดยการเลื่อน Slide Bar ด้านบน", #ข้อความใน card
                                className="card-text",
                            )
                        ]),#CardBody
                    ]
#Line Graph
card2 = [dbc.CardHeader(children=[
            dcc.Graph(id='everest-graph'),
                        ]),#CardHeader
            dbc.CardBody([
                html.P(
                    "กราฟเส้นนี้แสดงถึงจำนวนคนฆ่าตัวตายทั้งหมดในแต่ละปี", #ข้อความใน card
                    className="card-text",
                ),
                html.P(
                    "ซึ่งกราฟมีแนวโน้มเพิ่มขึ้นจากปี 1985 เรื่อยๆและคงที่", #ข้อความใน card
                    className="card-text",
                )
            ])
        ]
#Grouped Bar Chart
card3 = [dbc.CardHeader(children=[
                dcc.Graph(id='chocolatebar-graph'),
                        ]),#CardHeader
            dbc.CardBody([
                html.P(
                    "แนวโน้มในกราฟนี้จะแสดงถึงการฆ่าตัวตายว่าช่วงอายุระหว่าง 35-54 ปีได้มีการฆ่าตัวตายมากที่สุด", #ข้อความใน card
                    className="card-text",
                ),
                html.P(
                    "ซึ่งกราฟมีแนวโน้มเพิ่มขึ้นจากปี 1985 เรื่อยๆและคงที่", #ข้อความใน card
                    className="card-text",
                )
            ])
        ]
#Column แสดงข้อมูลประเทศที่มีจำนวนการฆ่าตัวตายมากที่สุด
mostdead = [
    dbc.Col(children=[
        html.H6("ประเทศที่มีการฆ่าตัวตายเยอะที่สุด"),
        html.H3("สหพันธรัฐรัสเซีย"),
        html.H2("{:,} คน".format(1209742))
    ])
]
#Column แสดงข้อมูลช่วงอายุที่มีจำนวนการฆ่าตัวตายมากที่สุด
agedead = [
    dbc.Col(children=[
        html.H6("ช่วงอายุที่มีการฆ่าตัวตายเยอะที่สุด"),
        html.H3("35-54 ปี"),
        html.H2("{:,} คน".format(2452141))
    ])
]
#Column แสดงข้อมูลจำนวนการฆ่าตัวตายในประเทศไทย
thaidead = [
    dbc.Col(children=[
        html.H6("ประเทศไทยมีการฆ่าตัวตาย"),
        html.H2("{:,} คน".format(110643))
    ])
]

app.layout = html.Div(children=[
            html.Div(
                id="header",
                className='jumbotron',
                children=[
                    html.H1(children="Data Visualization Dashboard"),
                    html.H3("แดชบอร์ดแสดงข้อมูลการฆ่าตัวตายของทั่วโลกตั้งแต่ปี 1985-2016"),
                    "เยี่ยมชม GitHub ของพวกเรา ",
                    dcc.Link(
                        href="https://github.com/artist0123/ProjectPSIT",
                        children=["กดที่นี่"],
                        )
                ]),#DivHeader
            dbc.Container(fluid=False,
            children=[#Container
                dbc.Row([#Row1
                    dbc.Col(mostdead),
                    dbc.Col(agedead),
                    dbc.Col(thaidead),
                ]),#Row1
                dbc.Row([#Row2
                    dbc.Col([#Col1
                        dbc.Card(card1, outline=True),
                        html.Br(),
                        dbc.Card(card2, outline=True),
                        html.Br(),
                        dbc.Card(card3, outline=True),
                    ])#Col1
            ])#Row2
        ]),#Container
    ])#Div1

In [14]:
# ส่วน Callback เพื่อใช้ในการ update graph
@app.callback(
    Output(component_id='world-choropleth', component_property='figure'),
    Output(component_id='everest-graph', component_property='figure'),
    Output(component_id='chocolatebar-graph', component_property='figure'),
    Input(component_id='year-slider', component_property='value')
)
def update_figure(year):
    df_choropleth = df_country_code.query('year=='+str(year))
    world_suicide = go.Figure(data=go.Choropleth(
        locations = df_choropleth['CODE'],
        z = df_choropleth['suicides_no'],
        text = df_choropleth['Country'],
        colorscale = 'Oryel',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='darkgray',
        marker_line_width=0.2,
        colorbar_title = 'Suicide Cases',
        ))

    world_suicide.update_layout(
    autosize=True,
    title_text='สถิติการฆ่าตัวตายของทั่วโลกในปี ' + str(year),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        )
    )
    everest_graph = px.line(df_year, x=df_year.index, y="suicides_no", title='สถิติการฆ่าตัวตายตั้งแต่ปี 1985 - 2016')

    chocolatebar_graph = px.bar(df_chocolate, x='age', y="suicides_no", color='sex', barmode='group', title='สถิติการฆ่าตัวตายโดยจำแนกเพศและช่วงอายุในปี 1985 - 2016')

    return world_suicide, everest_graph, chocolatebar_graph

In [15]:
# Run app
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
